In [1]:
class RO_Constraints(object):
    """This is the main class to produce a set of constraints for ROs"""
    def __init__(self, first_instance_names = 'design_1_i/LUT6_RO_0/inst/Inverter/LUT6_inst' \
                 ,other_instance_names = 'design_1_i/LUT6_RO_0/inst/notGates[@inst].Inverter/LUT6_inst'\
                 , num_stages = 3, shape = 1, lut_type = 'ALL', start_location = (0,0)):
        self.first_instance_names = first_instance_names
        self.other_instance_names = other_instance_names #@inst for instance number
        self.num_stages = num_stages
        self.shape = shape #num Slices in x directions
        self.lut_type = lut_type # A, B, C, D, All
        self.start_location = start_location #{generic_name : [num_RO, num_stages, location_tuple]} ; location_tuple = (x1, y1, x2, y2)
        self._outputfile = "constraints"
        
        if 'ALL' in self.lut_type:
            assert self.shape == - ( - self.num_stages // 4)
    def location(self):
        """For location constraints"""
        lut_types = ['A', 'B', 'C', 'D']
        x = self.start_location[0]
        y = self.start_location[1]
        str1 = 'set_property BEL '
        str2 = '6LUT [get_cells '
        str4 = 'set_property LOC SLICE_'
        with open(self._outputfile + ".XDC", "a") as file:
            for ROs in range(self.num_stages):
                if ROs == 0:
                    str3 = self.first_instance_names + ']'
                else:
                    str3 = '{' + self.other_instance_names.replace('@inst', str(ROs-1)) + '}' + ']'
                if 'ALL' in self.lut_type.upper():
                    lut_type = lut_types[ROs % 4]
                else:
                    lut_type = self.lut_type
                str5 = f'X{x}Y{y} [get_cells '
                file.write(str1 + lut_type + str2 + str3 + "\n")
                file.write(str4 + str5 + str3 + "\n")
                if 'ALL' in self.lut_type.upper():
                    y += ((ROs + 1 ) // 4)
                else:
                    y += int(self.shape / self.num_stages)    
    def loops(self):
        """For combinational loop constraints"""
        with open(self._outputfile + ".XDC", "a") as file:
            file.write("\n")
            for ROs in range(self.num_stages):
                file.write("set_property ALLOW_COMBINATORIAL_LOOPS true [get_nets {" +  self.other_instance_names.replace('@inst', str(ROs)) + '}' + ']'+ "\n")

The following will create one RO of five stages in one slice:

In [26]:
a = RO_Constraints(first_instance_names = f'design_1_i/LUT6_RO_0/inst/Inverter/LUT6_inst', \
                   other_instance_names = f'design_1_i/LUT6_RO_0/inst/notGates[@inst].Inverter/LUT6_inst',\
                   num_stages = 5, shape = 2, lut_type = "ALL" , start_location = (39,50))
a.location()

The following will create 4 ROs of five stages in exactly the same set of slices:

In [6]:
for RO in range(4):
    lut_types = ['A', 'B', 'C', 'D']
    a = RO_Constraints(first_instance_names = f'design_1_i/LUT6_RO_{RO}/inst/Inverter/LUT6_inst', \
                       other_instance_names = f'design_1_i/LUT6_RO_{RO}/inst/notGates[@inst].Inverter/LUT6_inst',\
                       num_stages = 5, shape = 5, lut_type = lut_types[RO % 4] , start_location = (59,50))
    a.location()

Adding combinational loop constraints for the heater:

In [3]:
a = RO_Constraints(first_instance_names = f'design_1_i/AXI4_heater_0/inst/SHEs[0].SHE/feedback', \
                   other_instance_names = f'design_1_i/AXI4_heater_0/inst/SHEs[@inst].SHE/feedback',\
                   num_stages = 480, shape = 120, lut_type = "ALL" , start_location = (39,50))
a.loops()